In [1]:
import numpy as np
import pandas as pd

from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

In [3]:
DATA_PATH = f"D:\code\DACON-Korean-sentence-relation-classification\open\open"
#train = pd.read_csv(os.path.join(DATA_PATH, "train_data.csv"))
#test = pd.read_csv(os.path.join(DATA_PATH, "test_data.csv") )
#submission = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))

In [7]:
max_len = 70
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

device = torch.device("cuda:0")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
model = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=False)

Downloading: 100%|██████████| 926/926 [00:00<00:00, 307kB/s]
Downloading: 100%|██████████| 243k/243k [00:00<00:00, 323kB/s]
Downloading: 100%|██████████| 483k/483k [00:00<00:00, 634kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 109kB/s]
Downloading: 100%|██████████| 422M/422M [00:23<00:00, 18.8MB/s]


In [4]:
df_test = pd.read_csv(os.path.join(DATA_PATH, "test_data.csv") )
X_test = df_test["premise"] + " " + tokenizer.sep_token + " " + df_test["hypothesis"]

In [5]:
y_preds = []
for i in tqdm(range(X_test.shape[0])):
    y_pred = model(X_test[i])[0]["label"].lower()
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

100%|██████████| 1666/1666 [01:04<00:00, 25.73it/s]


In [8]:
df_test["label"] = y_preds
df_submission = df_test.loc[:, ["index", "label"]]
df_submission.to_csv("./results/Huffon-klue-roberta-base-nli.csv", index=False)

In [9]:
import pandas as pd
import json

In [14]:
with open("D:\code\DACON-Korean-sentence-relation-classification\open\klue-nli-v1.1_dev.json", encoding='UTF8') as f:
    json_data = json.load(f)

In [21]:
df = pd.DataFrame(columns=["premise","hypothesis","label"])

In [23]:
df.index.names = ['index']

In [24]:
df

,premise,hypothesis,label
index,,,


In [25]:
for i in range(3000):
    df = df.append({"premise":json_data[i]['premise'],"hypothesis":json_data[i]['hypothesis'],"label":json_data[i]['gold_label']}, ignore_index=True)

In [27]:
df.index.names = ['index']

In [28]:
df

,premise,hypothesis,label
index,,,
0,흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.,어떤 방에서도 흡연은 금지됩니다.,contradiction
1,10명이 함께 사용하기 불편함없이 만족했다.,10명이 함께 사용하기 불편함이 많았다.,contradiction
2,10명이 함께 사용하기 불편함없이 만족했다.,성인 10명이 함께 사용하기 불편함없이 없었다.,neutral
3,10명이 함께 사용하기 불편함없이 만족했다.,10명이 함께 사용하기에 만족스러웠다.,entailment
4,10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요.,건물사람들은 수영장과 썬베드를 이용할 수 있습니다.,entailment
...,...,...,...
2995,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사는 중단없이 마무리 되었다.,contradiction
2996,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사가 중단된 건 세 번째이다.,neutral
2997,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선은 흔히 비자림로라고 불린다.,entailment


In [29]:
df.to_csv("klue_nli_dev.csv")

In [20]:
json_data[0]

{'guid': 'klue-nli-v1_dev_00000',
 'source': 'airbnb',
 'premise': '흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.',
 'hypothesis': '어떤 방에서도 흡연은 금지됩니다.',
 'gold_label': 'contradiction',
 'author': 'contradiction',
 'label2': 'contradiction',
 'label3': 'contradiction',
 'label4': 'contradiction',
 'label5': 'contradiction'}

In [16]:
df = pd.read_json(json_data)

ValueError: Invalid file path or buffer object type: <class 'list'>